In [6]:
%run Jansen_And_Rit.py
%run BOLD_Model.py
%run OptimiseFunctions.py

import numpy as np
import pickle
from skopt import gp_minimize
np.random.seed(123)

In [4]:
# Original A = 3.25 # excitatory, B = 22.0 # inhibitory, C = 135
# B > A. b < a. 
# b = 50, ad = 50, a = 100
# r_0, r_1, r_2 = 0.56
# mean_firing_threshold = 6
# max_firing_rate = 5

# A, B, C, a, ad, b, r_0, r_1, r_2, alpha, beta

# Alpha and beta 

# same sweep as https://doi.org/10.1371/journal.pcbi.1008737 - α 2 [0, 1], β 2 [0, 0.5] and r0 2 [0, 1].

eeg_res = gp_minimize(find_eeg_loss, [(2, 7), (15, 30), (80, 150), (50, 150), (25, 75), (25, 75), (0, 1), (0, 1), (0, 1), (0, 1), (0, 0.5)], n_jobs=-1, n_calls=100)

with open('eeg_res.pkl', 'wb') as f:
    pickle.dump(eeg_res, f)

6 21 128 122 50 64 0 1 0 0 0.3136585046688084
Creating RawArray with float64 data, n_channels=62, n_times=2000
    Range : 0 ... 1999 =      0.000 ...     1.999 secs
Ready.
Effective window size : 2.000 (s)
[0.64463955 0.62036123 0.68946285 0.63159735 0.75393816 0.60940926
 0.65395503 0.68418089 0.57766912 0.6261204  0.64325379 0.62193587
 0.61864887 0.70671656 0.62398683 0.68085997 0.64224285 0.62951686
 0.60905359 0.64494895 0.62839592 0.62511322 0.6057207  0.59706008
 0.60351749 0.62434311 0.6071372  0.60794899 0.70069582 0.68719399
 0.6367749  0.48982409 0.4671886  0.65714984 0.5792661  0.57074826
 0.57015934 0.55289062 0.63632227 0.63108305 0.63651417 0.64860173
 0.44981341 0.50230364 0.54343638 0.53412637 0.52713915 0.59446903
 0.60881408 0.61037878 0.61786048 0.50037659 0.49742872 0.47984114
 0.4707213  0.51717752 0.51631208 0.52584904 0.51351509 0.51771067
 0.50083811 0.52428581]
3 23 123 146 53 47 1 1 0 0 0.4482444841425226


KeyboardInterrupt: 

In [ ]:
bold_res = gp_minimize(find_bold_loss, [(2, 7), (15, 30), (80, 150), (50, 150), (25, 75), (25, 75), (0, 1), (0, 1), (0, 1), (0, 1), (0, 0.5)], n_jobs=-1, n_calls=100)

with open('bold_res.pkl', 'wb') as f:
    pickle.dump(bold_res, f)

print(get_cached_counts())